In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
#importing tensorflow
import tensorflow as tf 

In [ ]:
# checking version
tf.__version__

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
#Loading Facenet model

model = load_model('facenet_keras.h5')# load the model siamese trained model for facial recognition only

print(model.inputs)# input is 3Dimensional
print(model.outputs) #flatten output is 128

In [ ]:
#Multi-task Cascaded Convolutional Networks (MTCNN) is a framework developed as a solution for  face detection
# downloading the library to get bounding boxes around the faces in the images
!pip install mtcnn # 

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
# load image from file
image = plt.imread('pak_cric_group.jpg')
plt.figure(figsize=(10,6))
plt.imshow(image)

In [ ]:
from mtcnn.mtcnn import MTCNN
detector = MTCNN()
# detect faces in the image and storing it in results
results = detector.detect_faces(image)

In [ ]:
len(results) # 15 persons in the image

In [ ]:
results #bounding boxes for each face and position of eyes, nose and mouth

In [ ]:
#extract only face regions we can extract key['box from the results which is a dictionary']
faces=[]
for i in range(len(results)):
    # extract the bounding box from the first face
    x1, y1, width, height = results[i]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    # extract the face
    face = image[y1:y2, x1:x2]

    # resize pixels to the model size
    image1 = Image.fromarray(face)
    image1 = image1.resize((160, 160))
    face_array = np.array(image1)
    faces.append(face_array)

In [ ]:
len(faces)

In [ ]:
for i in faces:
    plt.imshow(i)
    plt.show()

In [ ]:
# images stored in database as refernce 
# calling one by one and getting bounding boxes
from os import listdir
folder = 'Database_pak/'
i = 1
face_ref=[]
# enumerate files
for filename in listdir(folder):

    path = folder + filename
    image = plt.imread(path)
    
    results = detector.detect_faces(image)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = image[y1:y2, x1:x2]
    # resize pixels to the model size
    image1 = Image.fromarray(face)
    image1 = image1.resize((160,160))
    face_array = np.array(image1)
    face_ref.append(face_array)
    
    plt.subplot(2, 7, i)
    plt.axis('off')
    plt.imshow(face)
    i += 1
plt.show()

In [ ]:
# getting embedding vector for the images in the database

In [ ]:
emb_ref=[]
for i in range(len(face_ref)):
    face_pn=face_ref[i].astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pn.mean(), face_pn.std()
    face_pn = (face_pn - mean) / std # z score,standard scaler

    face_pn = np.expand_dims(face_pn, axis=0)

    y_pn = model.predict(face_pn)
    # get embedding
    embedding_ref = y_pn[0] #128 vector flatten
    emb_ref.append(embedding_ref)

In [ ]:
len(emb_ref)

In [ ]:
emb_ref[0]

In [ ]:
i=9 #give the index of any player

plt.imshow(faces[i]) # Anchor/original face in the group image
face_anch=faces[i].astype('float32')
# standardize pixel values across channels (global)
mean, std = face_anch.mean(), face_anch.std()
face_anch = (face_anch - mean) / std

face_anch = np.expand_dims(face_anch, axis=0)

y_anch = model.predict(face_anch)
# get embedding
emb_anch = y_anch[0]

In [ ]:
emb_anch.shape

In [ ]:
#Compare the embedding of anchor image with reference image
res1=[]
for i in range(len(emb_ref)):
    res1.append(np.sqrt(np.sum((emb_ref[i]-emb_anch)**2)))

In [ ]:
res1

In [ ]:
len(res1)

In [ ]:
np.argmin(res1)

In [ ]:
plt.imshow(face_ref[np.argmin(res1)])